In [1]:
'''
        --STEPS--
1. Trimming Reads
2. Optical duplacates deliting
2.1. Pictures_1
3. Genome assembling
4. Blast to find organism
5. Contamination erasing
6. Reassembling
7. Annotation
8. Proteins
9. Binning
'''
pass

In [2]:
import os
import os.path
import pymongo

In [3]:
#input yours raw forward and reverse reads, files extentions available - *.fastq, *.fastq.gz, *.fq

raw_fastq_1 = "/mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_R1_001.fastq.gz"
raw_fastq_2 = "/mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_R2_001.fastq.gz"

In [4]:
EXECUTE_INDEX_BUILDING = False

if raw_fastq_1.endswith(".gz"):
    command = "gzip -d %s" % (raw_fastq_1)
    print(command)
    os.system(command)
    command = "gzip -d %s" % (raw_fastq_2)
    print(command)
    os.system(command)
    
    raw_fastq_1 = raw_fastq_1.replace(".gz", "")
    raw_fastq_2 = raw_fastq_2.replace(".gz", "")
    
if raw_fastq_1.endswith(".tar.gz"):
    command = "tar xfvz %s" % (raw_fastq_1)
    print(command)
    os.system(command)
    command = "tar xfvz %s" % (raw_fastq_2)
    print(command)
    os.system(command)
    
    raw_fastq_1 = raw_fastq_1.replace(".tar.gz", "")
    raw_fastq_2 = raw_fastq_2.replace(".tar.gz", "")
    
if raw_fastq_1.endswith(".fq"):
    new_raw_file_1 = raw_fastq_1.replace(".fq", ".fastq")
    command = "mv %s %s" % (raw_fastq_1, new_raw_file_1)
    print(command)
    os.system(command)
    new_raw_file_2 = raw_fastq_2.replace(".fq", ".fastq")
    command = "mv %s %s" % (raw_fastq_2, new_raw_file_2)
    print(command)
    os.system(command)
    raw_fastq_1 = new_raw_file_1
    raw_fastq_2 = new_raw_file_2
    
if raw_fastq_1.endswith("R1_001.fastq"):
    new_raw_file_1 = raw_fastq_1.replace("R1_001.fastq", "1.fastq")
    command = "mv %s %s" % (raw_fastq_1, new_raw_file_1)
    print(command)
    os.system(command)
    new_raw_file_2 = raw_fastq_2.replace("R2_001.fastq", "2.fastq")
    command = "mv %s %s" % (raw_fastq_2, new_raw_file_2)
    print(command)
    os.system(command)
    raw_fastq_1 = new_raw_file_1
    raw_fastq_2 = new_raw_file_2
    
if raw_fastq_1.endswith("_R1.fastq"):
    new_raw_file_1 = raw_fastq_1.replace("_R1.fastq", "_1.fastq")
    command = "mv %s %s" % (raw_fastq_1, new_raw_file_1)
    print(command)
    os.system(command)
    new_raw_file_2 = raw_fastq_2.replace("_R2.fastq", "_2.fastq")
    command = "mv %s %s" % (raw_fastq_2, new_raw_file_2)
    print(command)
    os.system(command)
    raw_fastq_1 = new_raw_file_1
    raw_fastq_2 = new_raw_file_2

assert raw_fastq_1.endswith("_1.fastq")
assert raw_fastq_2.endswith("_2.fastq")

gzip -d /mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_R1_001.fastq.gz
gzip -d /mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_R2_001.fastq.gz
mv /mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_R1_001.fastq /mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_1.fastq
mv /mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_R2_001.fastq /mnt/projects/dzilov/salmon14-182/reads/Salmon14-182_S4_L001_2.fastq


In [5]:
input1 = raw_fastq_1.split("/")
file_pref = input1[-1].split("_")
file1_pref = "_".join(file_pref[0:-1])
path_to_reads = "/".join(input1[0:-1])
print(file1_pref)
reads_folder = input1[-2]
print(reads_folder)
print(path_to_reads)

Salmon14-182_S4_L001
reads
/mnt/projects/dzilov/salmon14-182/reads


In [6]:
#SETTINGS AND TOOLS
# все частные штуки в сеттингсах нужно будет заменить на универсальные, для этого нужн отпарсить входные пути\файлы
# частные - такие как "Ecoli19-2_S2_L001_1.fastq", они не будут работать с другими файлами

#скачать все программы с работающими на момент создания скрипта в одну папку, чтобы пайп работал только с этими версиями

#составить список программ из конды и сделать аналог на снейкмейке

settings = {
    #START OPEARATIONS
    "trim_in_file_1" : raw_fastq_1,
    "trim_in_file_2" : raw_fastq_2,
    "trim_in_prefix" : raw_fastq_1.replace("_1.fastq", ""),
    "trim_out_prefix" : raw_fastq_1.replace("_1.fastq", ""),
    "trim_out_file_1" : raw_fastq_1.replace("_1.fastq", "") + ".trim_1.fastq",
    "trim_out_file_2" : raw_fastq_1.replace("_1.fastq", "") + ".trim_2.fastq",
    "rmdub_in_prefix" : raw_fastq_1.replace("_1.fastq", "") + ".trim",
    "rmdub_out_prefix" : raw_fastq_1.replace("_1.fastq", "") + ".rm",
    "rmdub_file_1" : raw_fastq_1.replace("_1.fastq", "") + ".rm_1.fastq",
    "rmdub_file_2" : raw_fastq_1.replace("_1.fastq", "") + ".rm_2.fastq",
    "fastqc_dir" : path_to_reads.replace(reads_folder, "") + "fastqc",
    "fastqc_file" : path_to_reads.replace(reads_folder, "") + "fastqc/" + file1_pref + "_1_fastqc.zip",
    #PICTURES
    "jelly_dir" : path_to_reads.replace(reads_folder, "") + "jellyfish",
    "jellycount_out_file" : path_to_reads.replace(reads_folder, "") + "jellyfish/" + file1_pref + ".jf2",
    "jellyhisto_out_file" : path_to_reads.replace(reads_folder, "") + "jellyfish/" + file1_pref + ".histo",
    #ASSEMBLY
    "spades_dir" :  path_to_reads.replace(reads_folder, "") + "spades",
    "spades_scaff" : path_to_reads.replace(reads_folder, "") + "spades" + "/scaffolds.fasta",
    "spades_cont" : path_to_reads.replace(reads_folder, "") + "spades" + "/contigs.fasta",
    "uni_dir" : path_to_reads.replace(reads_folder, "") + "unicycler",
    "uni_out" : path_to_reads.replace(reads_folder, "") + "unicycler" + "/assembly.fasta",
    # BLAST
    "blast_dir" : path_to_reads.replace(reads_folder, "") + "blast",
    "blast_db" : "/mnt/projects/shared/ncbi/blast/db/nt", #ШО ДЕЛАТЬ С ЭТИМ???
    "blastn_out" : path_to_reads.replace(reads_folder, "") + "blast" + "/" + file1_pref +".outfmt6",
    # FILTERING
    "filter_out" : path_to_reads.replace(reads_folder, "") + "blast" + "/" + file1_pref + ".best_single_hits.blastn",
    "index_file" : path_to_reads.replace(reads_folder, "") + "spades" + "/scaffolds.fasta.fai",
    "scaffolds_filtered" : path_to_reads.replace(reads_folder, "") + "spades" + "/scaffolds_filtered.fasta",
    "quast_dir" : path_to_reads.replace(reads_folder, "") + "quast",
    "quast_out" : path_to_reads.replace(reads_folder, "") + "quast/" + "report.txt",
    # ANNOTATION
    "prokka_dir" : path_to_reads.replace(reads_folder, "") + "prokka",
    # PROTEINS    
    "eggnog_path" : "/mnt/projects/vdikaya/Tools/eggnog-mapper-1.0.3/emapper.py", #при создании скрипта поменять на путь к файлу
    "eggnog_dir" : path_to_reads.replace(reads_folder, "") + "eggnog/",
    "eggnog_ann" : path_to_reads.replace(reads_folder, "") + "eggnog/" + "eggnog.emapper.annotations",
}

tools = {
    # START OPERATIONS
    "v2trim" : "/mnt/projects/shared/tools/V2_trim.exe %(trim_in_prefix)s %(trim_out_prefix)s 20 0 fastq /mnt/projects/shared/tools/illumina_ext.data",
    "rmdub" : "/mnt/projects/shared/tools/rmdup.exe %(rmdub_in_prefix)s %(rmdub_out_prefix)s 3000 3 0,30",
    # PICTURES
    "fastqc" : "fastqc -o %(fastqc_dir)s -t 32 %(trim_in_file_1)s %(trim_in_file_2)s",
    "jellycount" : "jellyfish count -m 23 -t 30 -s 2G -C -o %(jellycount_out_file)s %(trim_out_file_1)s %(trim_out_file_2)s",
    "jellyhisto" : "jellyfish histo -o %(jellyhisto_out_file)s %(jellycount_out_file)s",
    # ASSEMBLY
    "spades" : "spades -t 32 -1 %(rmdub_file_1)s -2 %(rmdub_file_2)s -o %(spades_dir)s",
    "unicycler" : "unicycler -1 %(rmdub_file_1)s -2 %(rmdub_file_2)s -o %(uni_dir)s -t 32 --mode normal",
    # BLAST
    "blastn" : "blastn -query %(spades_scaff)s -db %(blast_db)s -max_target_seqs 5 -outfmt 6 -evalue 1e-5 -num_threads 32 -out %(blastn_out)s",
    # FILTERING
    "filter" : "less %(blastn_out)s | sort  -k1,1 -k12,12nr -k11,11n | sort -u -k1,1 --merge > %(filter_out)s",
    "index" : "samtools faidx %(spades_scaff)s", 
    "quast" : "/mnt/projects/dzilov/soft/quast/quast-5.0.2/quast.py -o %(quast_dir)s %(scaffolds_filtered)s",
    # ANNOTATION
    "prokka" : "prokka --cpus 32 --outdir %(prokka_dir)s %(scaffolds_filtered)s",
    # PROTEINS
    "blastp" : """
    time blastp -db /mnt/projects/shared/ncbi/STRING/protein.sequences.v11.0.fa -query PROKKA_02052020.faa 
    -out PROKKA_02052020.string.blast -max_target_seqs 5 -outfmt "6 qseqid qgi qacc qaccver qlen sseqid sallseqid
    sgi sallgi sacc saccver sallacc slen qstart qend sstart send qseq sseq evalue bitscore score length pident nident
    mismatch positive gapopen gaps ppos frames qframe sframe btop staxids sscinames scomnames sblastnames
    sskingdoms stitle salltitles sstrand qcovs qcovhsp" -evalue 1e-5 -num_threads 32
    """,
    "eggnog" : "python2 %(eggnog_path)s -i %(prokka_faa)s --output %(eggnog_dir)s -m diamond --cpu 32 --report_orthologs",
}

In [7]:
# TESTED, WORKS
#1 STEP TRIMMING

command = tools["v2trim"] % settings
check_file = os.path.exists(settings["trim_out_file_1"])
if check_file == True:
    if os.path.getsize(settings["trim_out_file_1"]) > 0:
        print("U've already done Trimming")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done Trimming


In [8]:
# TESTED, WORKS
#2 STEP Optical duplacates deliting

assert os.path.exists(settings["trim_out_file_1"])
assert os.path.getsize(settings["trim_out_file_1"]) > 0

command = tools["rmdub"] % settings
check_file = os.path.exists(settings["rmdub_file_1"])
if check_file == True:
    if os.path.getsize(settings["rmdub_file_1"]) > 0:
        print("U've already done RMDUB")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done RMDUB


In [9]:
# TESTED, WORKS
#2.1. STEP PICTURES_1
#2.1.1. FASTQC OF RAW READS
assert os.path.exists(settings["trim_in_file_1"])
assert os.path.getsize(settings["trim_in_file_1"]) > 0

# CREATING DIRECTORIES

if os.path.exists(settings["fastqc_dir"]) == False:
    os.mkdir(settings["fastqc_dir"])
else:
    pass

# FASTQCSHING

command = tools["fastqc"] % settings
check_file = os.path.exists(settings["fastqc_file"])
if check_file == True:
    if os.path.getsize(settings["fastqc_file"]) > 0:
        print("U've already done FASTQC")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done FASTQC


In [10]:
# TESTED, WORKS
#2.1.2. JELLYFISH OF TRIMMED READS
assert os.path.exists(settings["trim_out_file_1"])
assert os.path.getsize(settings["trim_out_file_1"]) > 0

# CREATING DIRECTORIES

if os.path.exists(settings["jelly_dir"]) == False:
    os.mkdir(settings["jelly_dir"])
else:
    pass

# JELLYFISHING

command = tools["jellycount"] % settings
check_file = os.path.exists(settings["jellycount_out_file"])
if check_file == True:
    if os.path.getsize(settings["jellycount_out_file"]) > 0:
        print("U've already done JellyCount")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done JellyCount


In [11]:
# TESTED, WORKS
#2.1.3 JELLYFISH HISTOGRAM
assert os.path.exists(settings["jellycount_out_file"])
assert os.path.getsize(settings["jellycount_out_file"]) > 0

command = tools["jellyhisto"] % settings
check_file = os.path.exists(settings["jellyhisto_out_file"])
if check_file == True:
    if os.path.getsize(settings["jellyhisto_out_file"]) > 0:
        print("U've already done JellyHysto")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done JellyHysto


In [12]:
# TESTED, WORKS
#3 STEP GENOME ASSEMBLING (когда буду писать программу нужно задать в аргументах метод сборки, от него потом зависит бласт,
# по дефолту спадес)
#3.1. SPADES
assert os.path.exists(settings["rmdub_out_prefix"] + "_1.fastq")
assert os.path.getsize(settings["rmdub_out_prefix"] + "_1.fastq") > 0

# CREATING DIRECTORIES

if os.path.exists(settings["spades_dir"]) == False:
    os.mkdir(settings["spades_dir"])
else:
    pass

# SPADISHING

command = tools["spades"] % settings
check_file = os.path.exists(settings["spades_scaff"])
if check_file == True:
    if os.path.getsize(settings["spades_scaff"]) > 0:
        print("U've already done Spades")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done Spades


In [13]:
# TESTED, WORKS
#3.2. UNICYCLER
# conda install unicycler
# conda update --prefix /home/dzilov/soft/miniconda3 unicycler

assert os.path.exists(settings["rmdub_out_prefix"] + "_1.fastq")
assert os.path.getsize(settings["rmdub_out_prefix"] + "_1.fastq") > 0

# CREATING DIRECTORIES

if os.path.exists(settings["uni_dir"]) == False:
    os.mkdir(settings["uni_dir"])
else:
    pass

# UNICYCLISHING

command = tools["unicycler"] % settings
check_file = os.path.exists(settings["uni_out"])
if check_file == True:
    if os.path.getsize(settings["uni_out"]) > 0:
        print("U've already done Unicycler")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done Unicycler


In [14]:
# TESTED, WORKS
# 4 STEP BLAST

assert os.path.exists(settings["spades_scaff"])
assert os.path.getsize(settings["spades_scaff"]) > 0

# CREATING DIRECTORIES

if os.path.exists(settings["blast_dir"]) == False:
    os.mkdir(settings["blast_dir"])
else:
    pass

if os.path.exists(settings["blastn_out"]) == False:
    os.mknod(settings["blastn_out"])
else:
    pass
# BLASTISHING

command = tools["blastn"] % settings
check_file = os.path.exists(settings["blastn_out"])
if check_file == True:
    if os.path.getsize(settings["blastn_out"]) > 0:
        print("U've already done Blastn")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done Blastn


In [15]:
# TESTED, WORKS
# 5 STEP FILTERING

assert os.path.exists(settings["blastn_out"])
assert os.path.getsize(settings["blastn_out"]) > 0

command = tools["filter"] % settings
check_file = os.path.exists(settings["filter_out"])
if check_file == True:
    if os.path.getsize(settings["filter_out"]) > 0:
        print("U've already done Filtering")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done Filtering


In [16]:
# FASTA INDEXING

assert os.path.exists(settings["spades_scaff"])
assert os.path.getsize(settings["spades_scaff"]) > 0

command = tools["index"] % settings
check_file = os.path.exists(settings["index_file"])
if check_file == True:
    if os.path.getsize(settings["index_file"]) > 0:
        print("U've already done Indexing")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done Indexing


In [17]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
ncbi = myclient["NCBI"]
refids = ncbi.refids
taxids = ncbi.taxids

In [18]:
# STEP 6 CONTAMINATION ERASING

assert os.path.exists(settings["index_file"])
assert os.path.getsize(settings["index_file"]) > 0
        
bact_fai = settings["index_file"]
cont_to_len = {}
with open(bact_fai) as fh:
    for line in fh:
        a = line.strip().split("\t")
        cont_to_len[a[0]] = int(a[1])

In [19]:
assert os.path.exists(settings["filter_out"])
assert os.path.getsize(settings["filter_out"]) > 0

blast_file = settings["filter_out"]
cont_to_blast = {}
with open(blast_file) as fh:
    for line in fh:
        a = line.strip().split("\t")
        cont_to_blast[a[0]] = a[1]

In [20]:
from collections import Counter
count_names = Counter()
count_id = Counter()
total_length = 0
not_bacteria_node = []
for key, value in cont_to_blast.items():
    length = cont_to_len[key]
    val_split = value.split('.')[0]
    taxon_id = refids.find_one({"accession":val_split})["taxid"]
    taxon_name = taxids.find_one({"_id":taxon_id})["name"]
    taxon_name = " ".join(taxon_name.split()[:2])
    count_names[taxon_name] += length
    count_id[taxon_id] += length
    total_length += length
    if taxon_name != count_names.most_common()[0][0]:
        not_bacteria_node.append(key)

In [21]:
for taxon, size in count_names.most_common(100):
    print(taxon, "%s%%" % round(100.*size/total_length, 2))

Salmonella enterica 97.04%
Escherichia coli 2.53%
Klebsiella pneumoniae 0.4%
Bacillus cereus 0.02%
Cutibacterium acnes 0.01%
Psychrobacillus sp. 0.01%


In [22]:
bacteria_id = count_id.most_common()[0][0]
print("THE MOST COMMON ID IS :", bacteria_id)

THE MOST COMMON ID IS : 550537


In [23]:
print("NODES TO BE ERASED:", not_bacteria_node)
print(len(not_bacteria_node))

NODES TO BE ERASED: ['NODE_14_length_87187_cov_177.727452', 'NODE_18_length_34700_cov_347.014954', 'NODE_22_length_18653_cov_345.649628', 'NODE_33_length_665_cov_827.111524', 'NODE_36_length_487_cov_0.833333', 'NODE_38_length_475_cov_0.882184', 'NODE_40_length_459_cov_0.641566', 'NODE_41_length_455_cov_0.692073', 'NODE_43_length_430_cov_0.755776', 'NODE_54_length_128_cov_352.000000']
10


In [24]:
fasta = {}
header = None
with open(settings["spades_scaff"]) as fh:
    for i, line in enumerate(fh):
        line = line.strip()
        if line.startswith(">"):
            if header:
                fasta[header]="".join(seq)
            header = line
            seq = []
        else:
            seq.append(line)
if header:
    fasta[header] = "".join(seq)

In [25]:
i = 0
while i < len(not_bacteria_node) - 1:
    if ">" + not_bacteria_node[i] in fasta.keys():
            fasta.pop(">" + not_bacteria_node[i])
            i+=1

In [26]:
f = open(settings["scaffolds_filtered"], "w")
for key, value in fasta.items():
    f.write(key + "\n" + value + "\n")
f.close()

In [27]:
# Analize of assembly
# TESTED, WORKS
assert os.path.exists(settings["scaffolds_filtered"])
assert os.path.getsize(settings["scaffolds_filtered"]) > 0

# CREATING DIRECTORIES

if os.path.exists(settings["quast_dir"]) == False:
    os.mkdir(settings["quast_dir"])
else:
    pass

# QUASTING

command = tools["quast"] % settings
check_file = os.path.exists(settings["quast_out"])
if check_file == True:
    if os.path.getsize(settings["quast_out"]) > 0:
        print("U've already done QUAST")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done QUAST


In [28]:
# Analize of assembly
# TESTED, WORKS
assert os.path.exists(settings["scaffolds_filtered"])
assert os.path.getsize(settings["scaffolds_filtered"]) > 0

# PROKKING

command = tools["prokka"] % settings
check_file = os.path.exists(settings["prokka_dir"])
if check_file == True:
    if os.path.getsize(settings["prokka_dir"]) > 0:
        print("U've already done PROKKA")
        pass
    else:
        print(command)
        os.system(command)
else:
    print(command)
    os.system(command)

U've already done PROKKA


In [29]:
settings["prokka_faa"] = settings["prokka_dir"] + "/" + os.listdir(settings["prokka_dir"])[0].split(".")[0] + ".faa"

In [30]:
# Analize of assembly
# TESTED, WORKS
assert os.path.exists(settings["prokka_faa"])
assert os.path.getsize(settings["prokka_faa"]) > 0

# CREATING DIRECTORIES

if os.path.exists(settings["eggnog_dir"]) == False:
    os.mkdir(settings["eggnog_dir"])
else:
    pass

# EMAPPING

command = tools["eggnog"] % settings
check_file = os.path.exists(settings["eggnog_ann"])
if check_file == True:
    if os.path.getsize(settings["eggnog_ann"]) > 0:
        print("U've already done EGGNOG")
        pass
    else:
        print(command)
        #os.system(command)
else:
    print(command)
    #os.system(command)

U've already done EGGNOG


In [31]:
obo_annotation = "/mnt/projects/shared/go.obo"
settings["anno_file"] = settings["eggnog_ann"] + ".tsv"
settings["gene_copy_file"] = settings["eggnog_ann"] + ".gene_copy.tsv"

In [32]:
dict_GO = {}
header = None
with open (obo_annotation) as fh:
    for line in fh:
        line = line.strip()
        if line.startswith('id: GO') and header == None:
            header = line.replace('id: ', '')
            seq = []
            continue
        elif line.startswith('id: GO') and header != None:
            dict_GO[header] = " ".join(seq[:-2]).lower()
            header = line.replace('id: ', '')
            seq = []
        elif line.startswith("[Typedef]"):
            dict_GO[header] = " ".join(seq[:-2]).lower()
            break
        elif header != None:
            seq.append(line)

In [33]:
gene_of_interest = set()
with open(settings["eggnog_ann"]) as fh:
    for line in fh:
        if line.startswith('#'):
            continue
        line = line.strip().split('\t')
        gene_name = line[4]
        GO = line[5].split(',')
        for go in GO:
            if go in dict_GO and 'antigen'in dict_GO[go] and 'length' in dict_GO[go]:
                #print(go, gene_name, dict_GO[go])
                #break
                gene_of_interest.add(gene_name)

In [34]:
len(gene_of_interest)

0

In [35]:
print(gene_of_interest)

set()


In [36]:
from collections import defaultdict
go2genes = defaultdict(list)
with open(settings["anno_file"], "w") as fa:
    with open(settings["eggnog_ann"]) as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            line = line.strip().split("\t")  
            gene_name = line[4]
            GO = line[5].split(",")
            for go in GO:
                if go in dict_GO:
                    
                    if not line[4]:
                        line[4] = "NA"
                    gene_name = "%s:%s" % (line[0], line[4])
                    go2genes[go].append(gene_name)
    for go in go2genes.keys():
        fa.write("%s\t%s\t%s\n" % (go, ",".join(list(set(go2genes[go]))), dict_GO[go]))

In [37]:
gene_copy = defaultdict(int)
with open(settings["eggnog_ann"]) as fh:
    for line in fh:
        if line.startswith("#"):
            continue
        line = line.strip().split("\t")  
        if not line[4]:
            line[4] = "NA"
        else:
            gene_copy[line[4]] += 1


with open(settings["gene_copy_file"], "w") as fw:
    for gene_name, copy_number in gene_copy.items():
        fw.write("%s\t%s\n" % (gene_name, copy_number))